# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    create keyspace if not exists sparkify
    with replication =
    {
    'class' : 'SimpleStrategy' , 'replication_factor' : 1
    }
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Here, we will write a select query to obtain artist, song title and song length with a
filter on session id =338 and itemInSession = 4.
Thus, the CQL query would be:

select artist, song_title , song_len from songs_session where sessionId = 338 and itemInsession = 4;

Moreover, few things need to be noted here:

- the primary key will be comprised of sessionId as the partion key and itemInSession as clustering key to uniquely idetify the record.

Table Name: songs_session

Columns: artist, song_title, song_len, sessionId, itemInSession
    

In [10]:
query_1 = "create table if not exists songs_session(sessionId int, itemInSession int, artist text, song_title text, song_len float, primary key(sessionId, itemInSession))"

try:
    session.execute(query_1)
except Exception as e:
    print(e)

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "insert into songs_session (sessionId, itemInSession, artist, song_title, song_len)"
        query = query + "values(%s, %s, %s, %s, %s)"
        artist, user_firstname, gender, itemInSession, user_lastname, song_len, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist, song, float(song_len)))

#### Do a SELECT to verify that the data have been inserted into each table

###### select artist, song_title , song_len from songs_session where sessionId = 338 and itemInsession = 4

In [14]:
query = "select artist, song_title , song_len from songs_session where sessionId = 338 and itemInsession = 4"

T1 = PrettyTable(['Artist', 'Song_Title', 'Song_Length'])  #Created an object of Pretty Table with specified columns  

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    T1.add_row([row.artist, row.song_title, row.song_len])
print(T1)

+-----------+---------------------------------+--------------------+
|   Artist  |            Song_Title           |    Song_Length     |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


#### Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Here, we will write a select query to obtain artist, song, user first name, user last name with a filter on userid =10 and sessionid = 182 and the records are sorted according to itemInSession.

Thus, the CQL query would be:

select artist, song, user_firstname, user_lastname , itemInSession from users_songs where userId = 10 and sessionId = 182

Moreover, few things need to be noted here:

- We will have composite partition key of userId, sessionId.
- As we require the records to be sorted according to the itemInSession, thus it will act as a clutering key.

Table Name: users_songs

Columns: artist, song, user_firstname, user_lastname, itemInSession, userId


In [15]:
query_2 = "create table if not exists users_songs(userId int, sessionId int, itemInSession int, artist text, song text, user_firstname text,user_lastname text, primary key((userId, sessionId), itemInSession))"
try:
    session.execute(query_2)
except Exception as e:
    print(e)                  

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "Insert into users_songs (userId, sessionId, itemInSession, artist, song, user_firstname, user_lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        artist, user_firstname, gender, itemInSession, user_lastname, song_len, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), int(itemInSession), artist, song, user_firstname, user_lastname))

In [18]:
query = "select artist, song, user_firstname, user_lastname , itemInSession from users_songs where userId = 10 and sessionId = 182"

T2 = PrettyTable(['iteminsession', 'Artist', 'Song', 'Firstname', 'Lastname'])  #Created an object of Pretty Table with specified columns  

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    T2.add_row([row.iteminsession, row.artist, row.song, row.user_firstname, row.user_lastname])
print(T2)

+---------------+-------------------+------------------------------------------------------+-----------+----------+
| iteminsession |       Artist      |                         Song                         | Firstname | Lastname |
+---------------+-------------------+------------------------------------------------------+-----------+----------+
|       0       |  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|       1       |    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
|       2       | Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|       3       |   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+---------------+-------------------+------------------------------------------------------+-----------+----------+


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Here, we will write a select query to obtain first name, last name with a filter on song= 'All Hands Against His Own'.

Thus, the CQL query would be:

select user_firstname, user_lastname from music_app_history where song = 'All Hands Against His Own'

Moreover, few things need to be noted here:

- Here, first name and last name is not unique thus, we will add userId dimension as a clustering key to uniquely identify user names which are linked with the song mentioned.

- So, the primary key will consists of song(partition key) and userId(Clustering Key). 

Table Name: music_app_history

Columns: user_firstname, user_lastname, song, userId

### Drop the tables before closing out the sessions

In [20]:
query_3 = 'create table if not exists music_app_history(song text, userId int, user_firstname text, user_lastname text, primary key(song, userId))'

try:
    session.execute(query_3)
except Exception as e:
    print(e)

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "Insert into music_app_history (song, userId, user_firstname, user_lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        artist, user_firstname, gender, itemInSession, user_lastname, song_len, level, location, sessionId, song, userId = line
        session.execute(query, ( song, int(userId), user_firstname, user_lastname))

In [23]:
query = "select user_firstname, user_lastname from music_app_history where song = 'All Hands Against His Own'"

T3 = PrettyTable(['Firstname', 'Lastname'])  #Created an object of Pretty Table with specified columns 

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    T3.add_row([row.user_firstname, row.user_lastname])
print(T3)

+------------+----------+
| Firstname  | Lastname |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Droppping the before clossing the session

In [24]:
session.execute(""" drop table songs_session """)
session.execute(""" drop table users_songs """)
session.execute(""" drop table music_app_history """)

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()